In [1]:
from model.afnonet import AFNONet

In [34]:
from model.GradientModifier import *

In [4]:
import os 
import torch

$$
\left(
\begin{array}{ccccccc}
 \epsilon _{\text{II}}^1-\epsilon _{\text{II}}^{\text{real}} & 0 & 0 & 0 & 0 & 0 & 0 \\
 \epsilon _{\text{III}}^2-\epsilon _{\text{III}}^{\text{real}} & \epsilon _{\text{III}}^1-\epsilon _{\text{III}}^{\text{real}} & 0 & 0 & 0 & 0 & 0 \\
 \epsilon _{\text{IV}}^3-\epsilon _{\text{IV}}^{\text{real}} & \epsilon _{\text{IV}}^2-\epsilon _{\text{IV}}^{\text{real}} & \epsilon _{\text{IV}}^1-\epsilon _{\text{IV}}^{\text{real}} & 0 & 0 & 0 & 0 \\
 \epsilon _V^4-\epsilon _V^{\text{real}} & \epsilon _V^3-\epsilon _V^{\text{real}} & \epsilon _V^2-\epsilon _V^{\text{real}} & \epsilon _V^1-\epsilon _V^{\text{real}} & 0 & 0 & 0 \\
 \epsilon _{\text{VI}}^5-\epsilon _{\text{VI}}^{\text{real}} & \epsilon _{\text{VI}}^4-\epsilon _{\text{VI}}^{\text{real}} & \epsilon _{\text{VI}}^3-\epsilon _{\text{VI}}^{\text{real}} & \epsilon _{\text{VI}}^2-\epsilon _{\text{VI}}^{\text{real}} & \epsilon _{\text{VI}}^1-\epsilon _{\text{VI}}^{\text{real}} & 0 & 0 \\
 \epsilon _{\text{VII}}^6-\epsilon _{\text{VII}}^{\text{real}} & \epsilon _{\text{VII}}^5-\epsilon _{\text{VII}}^{\text{real}} & \epsilon _{\text{VII}}^4-\epsilon _{\text{VII}}^{\text{real}} & \epsilon _{\text{VII}}^3-\epsilon _{\text{VII}}^{\text{real}} & \epsilon _{\text{VII}}^2-\epsilon _{\text{VII}}^{\text{real}} & \epsilon _{\text{VII}}^1-\epsilon _{\text{VII}}^{\text{real}} & 0 \\
 \epsilon _{\text{VIII}}^7-\epsilon _{\text{VIII}}^{\text{real}} & \epsilon _{\text{VIII}}^6-\epsilon _{\text{VIII}}^{\text{real}} & \epsilon _{\text{VIII}}^5-\epsilon _{\text{VIII}}^{\text{real}} & \epsilon _{\text{VIII}}^4-\epsilon _{\text{VIII}}^{\text{real}} & \epsilon _{\text{VIII}}^3-\epsilon _{\text{VIII}}^{\text{real}} & \epsilon _{\text{VIII}}^2-\epsilon _{\text{VIII}}^{\text{real}} & \epsilon _{\text{VIII}}^1-\epsilon _{\text{VIII}}^{\text{real}} \\
\end{array}
\right)
$$

$$ 
f (X_t)+\epsilon_I^t=\hat{X_{t+1}} + \epsilon_I^t = X_{t+1}
$$

$$
f[f(X_t)] + \epsilon_{II}^t = X_{t+2}\\
f (X_{t+1})+\epsilon_I^{t+1}=\hat{X_{t+2}} + \epsilon_I^{t+1} = X_{t+2}
$$


$$
\epsilon_{II}^t = f (X_{t+1}) - f[f(X_t)]  +\epsilon_I^{t+1} = f (X_{t+1}) - f[\hat{X_{t+1}}]+\epsilon_I^{t+1}
$$

$$
\epsilon_{II}^t = \nabla f|_{X_{t+1}} [X_{t+1}-\hat{X_{t+1}}]+\epsilon_I^{t+1} =  \nabla f|_{X_{t+1}} \epsilon_I^{t} +\epsilon_I^{t+1}
$$

In [76]:
import numpy as np

In [123]:
def get_error_propagation(fourcastresult):
    error_order = torch.LongTensor([  0,
                                  2, 1,
                                  5, 4, 3,
                                  9, 8, 7, 6,
                                 14,13,12,11,10,
                                 20,19,18,17,16,15,
                                 27,26,25,24,23,22,21])
    N=7;x,y = torch.tril_indices(N,N)
    out={}
    for key in ['abs_error', 'est_error', 'Jacobianv', 'JacobV_N']:
        b = []
        for i in range(len(fourcastresult)-10):
            a = torch.zeros(N,N)
            a[x,y] = fourcastresult[i][key][error_order].sqrt()
            if key != 'JacobV_N':a=a/fourcastresult[i]['blevel_v'][:,None].sqrt()
            b.append(a)
        b = torch.stack(b)
        out[key] = b
    

    for key in ['alevel_v','blevel_v','JacobVA_N']:
        b = []
        for i in range(len(fourcastresult)-10):
            b.append(fourcastresult[i][key])
        b = torch.stack(b)
        out[key] = b
    return out

In [104]:
def get_fourcastresult(fourcastresult):
    #fourcastresult="checkpoints/WeathBench7066/AFNONet/ts_2_pretrain-2D706N_per_1_step/01_06_12_22-seed_73001"
    ROOT= fourcastresult
    fourcastresult_list = [os.path.join(ROOT,p) for p in os.listdir(fourcastresult) if 'fourcastresult.gpu' in p]
    fourcastresult={}
    for save_path in fourcastresult_list:
        tmp = torch.load(save_path)
        for key,val in tmp.items():
            if key not in fourcastresult:
                fourcastresult[key] = val
            else:
                if key == 'global_rmse_map':

                    fourcastresult['global_rmse_map'] = [a+b for a,b in zip(fourcastresult['global_rmse_map'],tmp['global_rmse_map'])]
                else:
                    fourcastresult[key] = val # overwrite
    return fourcastresult

In [135]:
fourcastresult="checkpoints/WeathBench7066/AFNONet/ts_2_pretrain-2D706N_per_1_step/01_06_12_22-seed_73001"
fourcastresult_pretrain = get_fourcastresult(fourcastresult)

In [134]:
fourcastresult_pretrain[0].keys()

dict_keys(['accu', 'rmse', 'std_pred', 'std_true', 'snap_line', 'hmse', 'abs_error', 'est_error', 'alevel_v', 'blevel_v', 'Jacobianv', 'JacobV_N', 'JacobVA_N'])

In [136]:
fourcastresult="checkpoints/WeathBench7066/AFNONet/ts_3_finetune-2D706N_per_1_step/01_09_02_41_31-seed_73001"
fourcastresult_finetune = get_fourcastresult(fourcastresult)

In [137]:
error_propagation_pretrain = get_error_propagation(fourcastresult_pretrain)
error_propagation_finetune = get_error_propagation(fourcastresult_finetune)

In [146]:
from mltool.visualization import *

tensor([[0.1116, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1576, 0.1149, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2322, 0.1994, 0.1335, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2976, 0.2763, 0.2361, 0.1538, 0.0000, 0.0000, 0.0000],
        [0.3664, 0.3508, 0.3248, 0.2794, 0.1849, 0.0000, 0.0000],
        [0.4360, 0.4236, 0.4040, 0.3742, 0.3221, 0.2143, 0.0000],
        [0.5070, 0.4965, 0.4809, 0.4582, 0.4238, 0.3637, 0.2429]])

In [178]:
error_propagation_finetune['est_error'].mean(0)

tensor([[0.2625, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2788, 0.2437, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3135, 0.2873, 0.2431, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3501, 0.3301, 0.2978, 0.2413, 0.0000, 0.0000, 0.0000],
        [0.3982, 0.3820, 0.3581, 0.3196, 0.2506, 0.0000, 0.0000],
        [0.4518, 0.4379, 0.4185, 0.3906, 0.3454, 0.2636, 0.0000],
        [0.5103, 0.4979, 0.4815, 0.4592, 0.4272, 0.3755, 0.2796]])

In [188]:
error_propagation_pretrain['alevel_v'].mean(0).sqrt()/error_propagation_finetune['blevel_v'].mean(0).sqrt()

tensor([0.8785, 0.8052, 0.7498, 0.7022, 0.6603, 0.6228, 0.5887])

In [182]:
error_propagation_pretrain['Jacobianv'].mean(0)

tensor([[0.5542, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.6753, 0.6637, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.7563, 0.7471, 0.7312, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.8206, 0.8132, 0.8006, 0.7801, 0.0000, 0.0000, 0.0000],
        [0.8786, 0.8722, 0.8621, 0.8462, 0.8201, 0.0000, 0.0000],
        [0.9336, 0.9279, 0.9191, 0.9064, 0.8863, 0.8535, 0.0000],
        [0.9874, 0.9821, 0.9743, 0.9632, 0.9473, 0.9220, 0.8816]])

In [186]:
error_propagation_pretrain['est_error'].mean(0)

tensor([[0.1116, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1576, 0.1149, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2322, 0.1994, 0.1335, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2976, 0.2763, 0.2361, 0.1538, 0.0000, 0.0000, 0.0000],
        [0.3664, 0.3508, 0.3248, 0.2794, 0.1849, 0.0000, 0.0000],
        [0.4360, 0.4236, 0.4040, 0.3742, 0.3221, 0.2143, 0.0000],
        [0.5070, 0.4965, 0.4809, 0.4582, 0.4238, 0.3637, 0.2429]])

In [189]:
error_propagation_finetune['Jacobianv'].mean(0)

tensor([[0.5603, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.6665, 0.6527, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.7407, 0.7300, 0.7138, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.8007, 0.7922, 0.7793, 0.7596, 0.0000, 0.0000, 0.0000],
        [0.8561, 0.8487, 0.8382, 0.8225, 0.7983, 0.0000, 0.0000],
        [0.9089, 0.9020, 0.8928, 0.8800, 0.8609, 0.8315, 0.0000],
        [0.9606, 0.9541, 0.9456, 0.9344, 0.9190, 0.8961, 0.8603]])

In [199]:
error_propagation_finetune['est_error'].mean(0)*error_propagation_finetune['blevel_v'].mean(0)[:,None].sqrt()/error_propagation_pretrain['blevel_v'].mean(0)[:,None].sqrt()

tensor([[0.2508, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2616, 0.2287, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2907, 0.2664, 0.2254, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3219, 0.3035, 0.2738, 0.2218, 0.0000, 0.0000, 0.0000],
        [0.3637, 0.3488, 0.3270, 0.2919, 0.2289, 0.0000, 0.0000],
        [0.4104, 0.3978, 0.3802, 0.3548, 0.3138, 0.2395, 0.0000],
        [0.4618, 0.4506, 0.4357, 0.4156, 0.3866, 0.3398, 0.2530]])

In [192]:
error_propagation_pretrain['blevel_v'].mean(0).sqrt()

tensor([0.2170, 0.2410, 0.2619, 0.2821, 0.3020, 0.3219, 0.3418])

In [191]:
error_propagation_finetune['blevel_v'].mean(0).sqrt()

tensor([0.2073, 0.2262, 0.2429, 0.2593, 0.2758, 0.2924, 0.3093])

In [171]:
x, y  = np.meshgrid(np.arange(7),np.arange(7))
z = error_propagation_finetune['abs_error'].mean(0).numpy()
data = np.stack([x,y,z],-1).reshape(-1,3)
data = data[data[:,2]>0]

In [204]:
weight = torch.load("checkpoints/WeathBench7066/AFNONet/time_step_2_pretrain-2D706N_every_1_step/10_14_20_18_11-seed_73001/backbone.best.pt", map_location='cpu')

In [206]:
[k for k in weight['model'] if 'patch_embed.proj' in k]

['patch_embed.proj.conv_layers.0.weight',
 'patch_embed.proj.conv_layers.1.weight']

In [70]:
error_propagation_finetune['JacobV_N'].mean(0)

tensor([[0.6446, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.7194, 0.7120, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.7791, 0.7752, 0.7665, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.8270, 0.8244, 0.8209, 0.8109, 0.0000, 0.0000, 0.0000],
        [0.8678, 0.8656, 0.8632, 0.8604, 0.8489, 0.0000, 0.0000],
        [0.9025, 0.9008, 0.8987, 0.8968, 0.8948, 0.8815, 0.0000],
        [0.9320, 0.9305, 0.9289, 0.9272, 0.9260, 0.9253, 0.9100]])

In [72]:
error_propagation_pretrain['Jacobianv'].mean(0)

tensor([[0.5542, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.6753, 0.6637, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.7563, 0.7471, 0.7312, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.8206, 0.8132, 0.8006, 0.7801, 0.0000, 0.0000, 0.0000],
        [0.8786, 0.8722, 0.8621, 0.8462, 0.8201, 0.0000, 0.0000],
        [0.9336, 0.9279, 0.9191, 0.9064, 0.8863, 0.8535, 0.0000],
        [0.9874, 0.9821, 0.9743, 0.9632, 0.9473, 0.9220, 0.8816]])

In [35]:
model = AFNONet(img_size=(32,64), patch_size=2, in_chans= 70 ,out_chans= 70 ,embed_dim=768,depth=12)
# x = torch.randn(2,70,32,64)
# y = torch.randn(2,70,32,64)
# torch.save(model.state_dict(),"debug/testmodel.pt")
# torch.save(x,"debug/testx.pt")
# torch.save(y,"debug/testy.pt")
model.load_state_dict(torch.load("debug/testmodel.pt"))
x = torch.load("debug/testx.pt")
y = torch.load("debug/testy.pt")

In [42]:
class NGmod_RotationDeltaX(Nodal_GradientModifier):
    def normed(self,a):
        shape = a.shape
        a = a.reshape(a.size(0),-1)
        a = a/a.norm(dim=1,keepdim=True)
        a = a.reshape(shape)
        return a 

    def getRotationDeltaloss(self, modelfun, x,  t , rotation_regular_mode = '0y0'):
        y = model(x)
        if rotation_regular_mode =='0y0':
            delta = (self.normed(y - x),)
        elif rotation_regular_mode =='0v0':
            delta = (self.normed(y.detach() - x),)
        elif rotation_regular_mode =='Yy0':
            delta = (self.normed(t - x),self.normed(y - x))
        elif rotation_regular_mode =='Yv0':
            delta = (self.normed(t - x),self.normed(y.detach() - x))
        elif rotation_regular_mode =='YyN':
            delta = (self.normed(t - x),self.normed(y - x),self.normed(torch.rand_like(x)))
        else:
            raise NotImplementedError
        penalty= 0
        for delta_cons in delta:
            grad = functorch.jvp(modelfun, (x,), (delta_cons,))[1] 
            position_range = list(range(1,len(grad.shape))) # (B,P, W,H ) --> (1,2,3)
            penalty       += ((torch.sum(grad**2,dim=position_range).sqrt()-1)**2).mean()
        return penalty

    def getL2loss(self,modelfun,x,chunk=10,coef=None):
        raise
    def getL1loss(self,modelfun,x,chunk=10,coef=None):
        raise

In [8]:
from einops.layers.torch import Rearrange

In [9]:
import torch.nn as nn

In [10]:
class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(30,30)
    def forward(self,x):
        x = self.linear(x)
        x = torch.sigmoid(x)
        x = x**2
        return x
model = MyModel()
torch.save(model.state_dict(),"debug/testmodel2.pt")
x = torch.randn(2,30)
y = torch.randn(2,30)
torch.save(x,"debug/testx2.pt")
torch.save(y,"debug/testy2.pt")

In [11]:
model = AFNONet(img_size=(32,64), patch_size=2, in_chans= 70 ,out_chans= 70 ,embed_dim=768,depth=12)
# x = torch.randn(2,70,32,64)
# y = torch.randn(2,70,32,64)
# torch.save(model.state_dict(),"debug/testmodel.pt")
# torch.save(x,"debug/testx.pt")
# torch.save(y,"debug/testy.pt")
model.load_state_dict(torch.load("debug/testmodel.pt"))
x = torch.load("debug/testx.pt")
y = torch.load("debug/testy.pt")
ngmod = NGmod_RotationDeltaX(0,0,0 )

In [16]:
cotangents1 = torch.randn(3,*x.shape)

In [43]:
model = AFNONet(img_size=(32,64), patch_size=2, in_chans= 70 ,out_chans= 70 ,embed_dim=768,depth=12)
# x = torch.randn(2,70,32,64)
# y = torch.randn(2,70,32,64)
# torch.save(model.state_dict(),"debug/testmodel.pt")
# torch.save(x,"debug/testx.pt")
# torch.save(y,"debug/testy.pt")
model.load_state_dict(torch.load("debug/testmodel.pt"))
x = torch.load("debug/testx.pt")
y = torch.load("debug/testy.pt")
ngmod = NGmod_RotationDeltaX(0,0,0 )
loss = ngmod.getRotationDeltaloss(model,x, y,rotation_regular_mode='YyN')
print(loss)
loss.backward()
ll = [[name,p.grad.norm()] for idx,(name,p) in enumerate(model.named_parameters()) if p.grad is not None]
for i in range(-10,-1,1):
    name,pnorm = ll[i] 
    print(f"{name} - {pnorm}")
    

tensor(1.8809, grad_fn=<AddBackward0>)
blocks.11.mlp.fc1.weight - 0.12911739945411682
blocks.11.mlp.fc1.bias - 0.003467740025371313
norm.weight - 0.028279943391680717
norm.bias - 0.0017165266908705235
pre_logits.conv1.weight - 2.469726085662842
pre_logits.conv1.bias - 0.0033190883696079254
pre_logits.conv2.weight - 1.0886025428771973
pre_logits.conv2.bias - 0.002785270567983389
head.weight - 0.2757563292980194


In [91]:
model = AFNONet(img_size=(32,64), patch_size=2, in_chans= 70 ,out_chans= 70 ,embed_dim=768,depth=12)
# x = torch.randn(2,70,32,64)
# y = torch.randn(2,70,32,64)
# torch.save(model.state_dict(),"debug/testmodel.pt")
# torch.save(x,"debug/testx.pt")
# torch.save(y,"debug/testy.pt")
model.load_state_dict(torch.load("debug/testmodel.pt"))
x = torch.load("debug/testx.pt")
y = torch.load("debug/testy.pt")
ngmod = NGmod_RotationDelta2(0,0,0 )
loss = ngmod.getRotationDeltaloss(model,x, model(x), y,rotation_regular_mode='0y0')
print(loss)
loss.backward()
ll = [[name,p.norm()] for idx,(name,p) in enumerate(model.named_parameters()) if p.grad is not None]
for i in range(-10,-1,1):
    name,pnorm = ll[i] 
    print(f"{name} - {pnorm}")
    

tensor(0.9506, grad_fn=<MeanBackward0>)
blocks.11.mlp.fc1.weight - 30.728208541870117
blocks.11.mlp.fc1.bias - 0.0
norm.weight - 27.712812423706055
norm.bias - 0.0
pre_logits.conv1.weight - 15.994429588317871
pre_logits.conv1.bias - 0.19202826917171478
pre_logits.conv2.weight - 19.3148250579834
pre_logits.conv2.bias - 0.1864301562309265
head.weight - 9.63589096069336


In [82]:
model = MyModel()
model.load_state_dict(torch.load("debug/testmodel2.pt"))
x = torch.load("debug/testx2.pt")
y = torch.load("debug/testy2.pt")
ngmod = NGmod_RotationDelta(0,0,0 )
loss = ngmod.getRotationDeltaloss(model,x,model(x),y,rotation_regular_mode='0y0')
print(loss)
loss.backward()
for idx,(name,p) in enumerate(model.named_parameters()):
    print(f"{name} - {p.grad.norm()}")
    

tensor(0.0157, grad_fn=<MeanBackward0>)
linear.weight - 0.047377560287714005
linear.bias - 0.004780657589435577


In [81]:
model = MyModel()
model.load_state_dict(torch.load("debug/testmodel2.pt"))
x = torch.load("debug/testx2.pt")
y = torch.load("debug/testy2.pt")
ngmod = NGmod_RotationDelta2(0,0,0 )
loss = ngmod.getRotationDeltaloss(model,x,model(x),y,rotation_regular_mode='0y0')
print(loss)
loss.backward()
for idx,(name,p) in enumerate(model.named_parameters()):
    print(f"{name} - {p.grad.norm()}")
    

tensor(0.9756, grad_fn=<MeanBackward0>)
linear.weight - 0.0752129852771759
linear.bias - 0.005585702136158943


In [78]:
class NGmod_RotationDelta2(Nodal_GradientModifier):
    def normed(self,a):
        shape = a.shape
        a = a.reshape(a.size(0),-1)
        a = a/a.norm(dim=1,keepdim=True)
        a = a.reshape(shape)
        return a 

    def getRotationDeltaloss(self, modelfun, x, y, t , rotation_regular_mode = '0y0'):
        #y, vjpfunc = functorch.vjp(modelfun, x) # notice this will calculate f(x) again, so can be reduced in real implement.
        if rotation_regular_mode =='0y0':
            delta = self.normed(y - x)
        elif rotation_regular_mode =='0v0':
            delta = self.normed(y.detach() - x)
        elif rotation_regular_mode =='Yy0':
            delta = torch.cat([self.normed(t - x),self.normed(y - x)])
        elif rotation_regular_mode =='Yv0':
            delta = torch.cat([self.normed(t - x),self.normed(y.detach() - x)])
        elif rotation_regular_mode =='YyN':
            delta = torch.cat([self.normed(t - x),self.normed(y - x),self.normed(torch.rand_like(x))])
        else:
            raise NotImplementedError
        #_,vjpfunc = functorch.vjp(modelfun, x) # notice this will calculate f(x) again, so can be reduced in real implement.    
        #grad = vjpfunc(delta)[0]
        grad           = functorch.jvp(model, (x,), (delta,))[1]
        position_range = list(range(1,len(grad.shape))) # (B,P, W,H ) --> (1,2,3)
        penalty        = ((torch.sum(grad**2,dim=position_range)-1)**2).mean()
        return penalty